# Encoder Decoder 

### Imports

In [20]:
import pandas as pd
import numpy as np
import re
import string

### Importing Data

In [3]:
df = pd.read_csv('ASL_English.csv')
df.head()

,English,ASL Gloss
0,Do you want a ride to the mall?,M-A-L-L RIDE WANT YOU Q
1,Yes I want to start buying Christmas gifts soon,YES SOON CHRISTMAS GIFTS START BUYING WANT ME
2,Please sit in this chair,THIS CHAIR PLEASE SIT
3,I like to fly small planes,SMALL PLANES FLY LIKE ME
4,He will go later,HE GO WILL


In [10]:
# AVG/Mode in ASL Gloss 
# 18 characters
df['ASL Gloss'].str.len().mode()

0    18
Name: ASL Gloss, dtype: int64

### Preprocessing

Note:
- Replace the numbers/digits
- Check regarding Finger spellings
- Check if it is required to add start and end tokens to target sequences

In [ ]:
# Remove extra spaces
df['English'] = df['English'].apply(lambda x: x.strip())
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: x.strip())

# Lowercase all characters
df['English'] = df['English'].apply(lambda x: x.lower())
df['ASL Gloss'] = df['ASL Gloss'].apply (lambda x: x.lower())

In [ ]:
# Remove quotes # Might not need this
# df['English'] = df['English'].apply(lambda x: re.sub (r"'", '', x))
# df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: re.sub (r"'", '', x))

In [ ]:
# Remove all special character
df['English'] = df['English'].apply(lambda x: ' '.join (ch for ch in x if ch not in set(string.punctuation)))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: ' '.join (ch for ch in x if ch not in set(string.punctuation)))

**Replace them later** and remove all numbers/digits

In [18]:
# Check if dataset has numbers
print(df['English'].str.contains(r'\d').any())
print(df['ASL Gloss'].str.contains(r'\d').any())

True
True


In [31]:
replacements = {'1': "one", '2':"two", '3':"three", '4':"four", '5':"five", '6':"six", '7':"seven", '8':"eight", '9':"nine", '0':"zero"}
df['English'] = df['English'].apply(lambda x: re.sub('(\d)', lambda m: replacements[m.group()], x))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: re.sub('(\d)', lambda m: replacements[m.group()], x))

'one-seven-one A-N-Y-W-H-E-R-E L-A-N-E nine-eight-seven-six-five'

In [34]:
# Add tokens to target sequence
df['English'] = df['English'].apply(lambda x : 'START_ ' + x + ' _END')

0            START_ Do you want a ride to the mall? _END
1      START_ Yes I want to start buying Christmas gi...
2                   START_ Please sit in this chair _END
3                 START_ I like to fly small planes _END
4                           START_ He will go later _END
                             ...                        
272                    START_ Sit in the wheelchair _END
273                        START_ You have a sprain _END
274          START_ You need to get to the hospital _END
275            START_ Im dizzy and my stomach hurts _END
276                        START_ Well go to the ER _END
Name: English, Length: 277, dtype: object

In [35]:
# Get English and ASL Vocabulary
all_eng_words = set()

for eng in df ['English'] :
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_ASL_words = set()

for asl in df ['ASL Gloss'] :
    for word in asl.split():
        if word not in all_ASL_words:
            all_ASL_words.add(word)

In [39]:
print("English Words:", len(all_eng_words))
print("ASL Words:", len(all_ASL_words))

English Words: 575
ASL Words: 478


In [46]:
input_words = sorted(list(all_ASL_words))
target_words = sorted(list(all_eng_words) )
num_encoder_tokens = len(all_ASL_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(478, 575)

### Encoder-Decoder Architecture

In [45]:
from keras.layers import Dropout
from keras.layers import Input, LSTM, Embedding, Dense

latent_dim = 64

# Encoder
encoder_inputs = Input(shape=(None, ) )
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero = True) (encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# Discard 'encoder outputs' and only keep the states.
encoder_states = [state_h, state_c ]

NameError: name 'Input' is not defined

In [ ]:
# Set up the decoder, using 'encoder states' as initial state.
decoder inputs Input (shape= (None, ) )
dec emb layer = Embedding (num decoder tokens, latent dim, mask zero True)
dec emb = dec emb layer (decoder inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder 1 stm = LSTM (latent dim, return sequences—Truet return state—True)
decoder outputs,
= decoder 1 stm (dec em-b,
initial state—encoder states)
decoder dense = Dense (num decoder tokens, activation—' softmax' )
decoder outputs decoder dense (decoder outputs)
# Define the model that will turn
# s encoder input data' s 'decoder input datas into 'decoder target data
model Model ( [encoder inputs, decoder inputs] , decoder outputs)
model. compile (optimizer—' rmsprop', loss—' categorical crossentropy' )